<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [ ]:
!pip install -q spacy-stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# para stop words
import stanza
import spacy_stanza
stanza.download("es")
nlp = spacy_stanza.load_pipeline("es")

INFO:stanza:Downloading default packages for language: es (Spanish) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: es (Spanish):
| Processor    | Package  |
---------------------------
| tokenize     | ancora   |
| mwt          | ancora   |
| pos          | ancora   |
| lemma        | ancora   |
| constituency | combined |
| depparse     | ancora   |
| sentiment    | tass2020 |
| ner          | conll02  |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

import os
import platform

import re
import unicodedata
import string
import spacy


### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [ ]:
urlsSubtitulosADescargar = [
    'https://subs.argenteam.net/Star.Wars(1977)-aRGENTeaM-55.zip', # IV
    'https://subs.argenteam.net/Star.Wars.Episode.V(1980)-aRGENTeaM-30898.zip', # V
    'https://subs.argenteam.net/Star.Wars.Episode.VI(1983)-aRGENTeaM-30903.zip', # VI
    # 'https://subs.argenteam.net/Star.Wars.Episode.I(1999)-aRGENTeaM-30889.zip', # I
    # 'https://subs.argenteam.net/Star.Wars.Episode.II(2002)-aRGENTeaM-30893.zip', # II
    # 'https://subs.argenteam.net/Star.Wars.Episode.III(2005)-aRGENTeaM-3597.zip', # III
    # 'https://subs.argenteam.net/Star.Wars.Episode.VII-The.Force.Awakens.(2015)-aRGENTeaM-60426.zip', # VII
    # 'https://subs.argenteam.net/Star.Wars.Episode.VIII-The.Last.Jedi.(2017)-aRGENTeaM-71696.zip', # VIII
    # 'https://subs.argenteam.net/Star.Wars.Episode.IX-The.Rise.of.Skywalker.(2019)-aRGENTeaM-80480.zip', # IX
]

In [ ]:
# Descargar la carpeta de dataset
if os.access('./subs_dataset', os.F_OK) is False:
  # descargo los subtitulos
  for url in urlsSubtitulosADescargar:
    print(url)
    # obtengo nombre documento
    nombreDocumento = url.split('/')[-1]
    print(nombreDocumento)
    # descargo como .zip
    !wget {'"'+ url +'"'}
    # descomprimo
    !unzip {'"'+ nombreDocumento +'"'} -d './subs_dataset'
else:
    print("El dataset ya se encuentra descargado")

https://subs.argenteam.net/Star.Wars(1977)-aRGENTeaM-55.zip
Star.Wars(1977)-aRGENTeaM-55.zip
--2023-05-16 01:51:23--  https://subs.argenteam.net/Star.Wars(1977)-aRGENTeaM-55.zip
Resolving subs.argenteam.net (subs.argenteam.net)... 172.67.206.63, 104.21.58.185, 2606:4700:3037::ac43:ce3f, ...
Connecting to subs.argenteam.net (subs.argenteam.net)|172.67.206.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39169 (38K) [application/zip]
Saving to: ‘Star.Wars(1977)-aRGENTeaM-55.zip’

Star.Wars(1977)-aRG 100%[===================>]  38.25K  --.-KB/s    in 0.001s  

2023-05-16 01:51:23 (49.3 MB/s) - ‘Star.Wars(1977)-aRGENTeaM-55.zip’ saved [39169/39169]

Archive:  Star.Wars(1977)-aRGENTeaM-55.zip
  inflating: ./subs_dataset/Star.Wars.Episode.IV.A.New.Hope.DVDRip.XviD-DoNE.CD1.srt  
  inflating: ./subs_dataset/Star.Wars.Episode.IV.A.New.Hope.DVDRip.XviD-DoNE.CD2.srt  
https://subs.argenteam.net/Star.Wars.Episode.V(1980)-aRGENTeaM-30898.zip
Star.Wars.Episode.V(1980)-aR

In [ ]:
# Posibles bandas
srtALeer = os.listdir("./subs_dataset/")

In [ ]:
# lee el primer subtitulo
df = pd.read_csv("./subs_dataset/" + srtALeer[0], sep='/n', header=None, encoding='latin-1')
# elimina lineas con '-->' y la superior
df.drop(df[df[0].str.contains('-->')].index - 1 , inplace = True) #quita numeros
df.drop(df[df[0].str.contains('-->')].index , inplace = True) #quita timepos
df.drop(df[df[0].str.contains('www.argenteam.net')].index , inplace = True) #quita marcas
df.drop(df[df[0].str.contains('Subtítulos por aRGENTeaM')].index , inplace = True) #quita marcas

# lee el resto y los agrega
for srlFile in srtALeer[1:]:
  # lee srt
  dfTemp = pd.read_csv("./subs_dataset/" + srlFile, sep='/n', header=None, encoding='latin-1')

  # elimina lineas con '-->' y la superior
  dfTemp.drop(dfTemp[dfTemp[0].str.contains('-->')].index - 1 , inplace = True) #quita numeros
  dfTemp.drop(dfTemp[dfTemp[0].str.contains('-->')].index , inplace = True) #quita timepos
  dfTemp.drop(dfTemp[dfTemp[0].str.contains('www.argenteam.net')].index , inplace = True) #quita marcas
  dfTemp.drop(dfTemp[dfTemp[0].str.contains('Subtítulos por aRGENTeaM')].index , inplace = True) #quita marcas
  # los agrega al df
  df = df.append(dfTemp, ignore_index=False)

df.shape


<ipython-input-7-ef1d2fe254c0>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("./subs_dataset/" + srtALeer[0], sep='/n', header=None, encoding='latin-1')
<ipython-input-7-ef1d2fe254c0>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dfTemp = pd.read_csv("./subs_dataset/" + srlFile, sep='/n', header=None, encoding='latin-1')
<ipython-input-7-ef1d2fe254c0>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dfTemp, ignore_index=False)
<ipython-input-7-ef1d2fe254c0>:12: ParserWarning: 

(4957, 1)

In [ ]:
df.head(10)

,0
2,¡Esto no es una votación!
5,No puedes saltar a la velocidad luz
6,en este campo de asteroides.
9,"Siéntate, preciosa."
10,Vamos a despegar.
13,- ¡Mire!
14,- Ya lo veo.
17,- ¡Estamos acabados!
18,- La cueva se cierra.
21,- Esto no es una cueva.


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 4957


In [ ]:
# El preprocesamento en castellano requiere más trabajo

# Referencia de regex:
# https://docs.python.org/3/library/re.html

def preprocess_clean_text(text):
    # sacar tildes de las palabras:
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # quita los <i> e </i>
    patron = r'<\/?i>'  # Expresión regular para encontrar <i> o </i>
    text = re.sub(patron, '', text)  # Reemplazar las ocurrencias de <i> y </i> con una cadena vacía
    # quita las stopWords

    # quitar caracteres especiales
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]' # [^ : ningún caracter de todos estos
    # (termina eliminando cualquier caracter distinto de los del regex)
    text = re.sub(pattern, '', text)
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' # igual al anterior pero sin cifras numéricas
    # quitar números
    text = re.sub(pattern, '', text)
    # quitar caracteres de puntuación
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

In [ ]:
# limpia cada frace del dataframe
df[0] = df[0].apply( lambda x: (preprocess_clean_text( x )) )
df.head(10)

,0
2,Esto no es una votacion
5,No puedes saltar a la velocidad luz
6,en este campo de asteroides
9,Sientate preciosa
10,Vamos a despegar
13,Mire
14,Ya lo veo
17,Estamos acabados
18,La cueva se cierra
21,Esto no es una cueva


### 1 - Preprocesamiento

In [ ]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:2]

[['esto', 'no', 'es', 'una', 'votacion'],
 ['no', 'puedes', 'saltar', 'a', 'la', 'velocidad', 'luz']]

In [ ]:
spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS
print('Number of stop words: %d' % len(spacy_stopwords))
print('First ten stop words: %s' % list(spacy_stopwords)[:20])
# quita las stopWords
for i in range(0,len(sentence_tokens)):
  array = []
  for word in sentence_tokens[i]:
    if word not in spacy_stopwords:
      array.append(word)
  sentence_tokens[i] = array



Number of stop words: 521
First ten stop words: ['ahí', 'nueve', 'propios', 'hacerlo', 'ella', 'peor', 'estais', 'tuyas', 'allí', 'éste', 'anterior', 'todavia', 'unas', 'aun', 'uno', 'total', 'de', 'para', 'existe', 'con']


In [ ]:
# Demos un vistazo
sentence_tokens[:2]

[['votacion'], ['puedes', 'saltar', 'velocidad', 'luz']]

### 2 - Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=50,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 4957


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.key_to_index ))

Cantidad de words distintas en el corpus: 372


### 3 - Entrenar el modelo generador

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 24762.994140625
Loss after epoch 1: 25975.037109375
Loss after epoch 2: 24910.5625
Loss after epoch 3: 22364.359375
Loss after epoch 4: 16265.4140625
Loss after epoch 5: 10063.875
Loss after epoch 6: 7929.4921875
Loss after epoch 7: 7282.828125
Loss after epoch 8: 7045.71875
Loss after epoch 9: 7065.484375
Loss after epoch 10: 6741.59375
Loss after epoch 11: 6983.59375
Loss after epoch 12: 6947.21875
Loss after epoch 13: 6864.578125
Loss after epoch 14: 6730.34375
Loss after epoch 15: 6848.46875
Loss after epoch 16: 6797.828125
Loss after epoch 17: 6654.125
Loss after epoch 18: 7145.484375
Loss after epoch 19: 7056.953125
Loss after epoch 20: 6688.015625
Loss after epoch 21: 6801.21875
Loss after epoch 22: 6702.828125
Loss after epoch 23: 6749.671875
Loss after epoch 24: 6603.21875
Loss after epoch 25: 6338.40625
Loss after epoch 26: 6471.03125
Loss after epoch 27: 6529.5
Loss after epoch 28: 6352.15625
Loss after epoch 29: 6380.40625
Loss after epoch 30: 6609.90625

(334029, 866500)

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["yoda"], topn=10)

[('prometo', 0.8678575754165649),
 ('debo', 0.8575572967529297),
 ('presencia', 0.8433343768119812),
 ('maestro', 0.8388802409172058),
 ('sentido', 0.835220217704773),
 ('lejos', 0.8259782791137695),
 ('volver', 0.8257198333740234),
 ('encontrar', 0.8104009628295898),
 ('queda', 0.8086608052253723),
 ('ud', 0.8067443370819092)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(positive=["estrella"], topn=10)

[('sistemas', 0.9782299995422363),
 ('siento', 0.895727813243866),
 ('minutos', 0.8794825077056885),
 ('queda', 0.8548435568809509),
 ('lejos', 0.8235612511634827),
 ('planos', 0.8036399483680725),
 ('laser', 0.7691442370414734),
 ('armas', 0.7322741746902466),
 ('yoda', 0.7272737622261047),
 ('operacional', 0.7067357301712036)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["luke","yoda"], topn=10)

[('sentido', 0.869152307510376),
 ('volver', 0.8491125106811523),
 ('aprender', 0.8471088409423828),
 ('entrenamiento', 0.8465972542762756),
 ('destino', 0.8460654020309448),
 ('preocupes', 0.8460564613342285),
 ('debo', 0.8369970321655273),
 ('digas', 0.8311010599136353),
 ('chewbacca', 0.8296388983726501),
 ('volvere', 0.8290704488754272)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darth","vader","hijo"], negative=[], topn=10)

[('trampa', 0.949791669845581),
 ('padre', 0.9370139837265015),
 ('dijiste', 0.8820943832397461),
 ('gracias', 0.8788265585899353),
 ('llegado', 0.8787726759910583),
 ('aprender', 0.8731719255447388),
 ('lord', 0.8582790493965149),
 ('sentimientos', 0.8554969429969788),
 ('hombres', 0.853635311126709),
 ('entrenamiento', 0.8516610860824585)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["luke","darth","vader"], topn=5)

[('trampa', 0.9408879280090332),
 ('padre', 0.9068341255187988),
 ('aprender', 0.8945216536521912),
 ('gracias', 0.869488000869751),
 ('dijiste', 0.8681919574737549)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(negative=["luke"], topn=5)

[('manos', 0.2937866747379303),
 ('capaz', 0.18617717921733856),
 ('escudos', 0.17612911760807037),
 ('torre', 0.026434917002916336),
 ('dios', 0.025364069268107414)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["skywalker","yoda"], topn=5)

[('jedi', 0.8899532556533813),
 ('aprendiz', 0.8772623538970947),
 ('fuerza', 0.8533816933631897),
 ('entrenamiento', 0.8490281701087952),
 ('sentido', 0.8417330980300903)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["fuerza"])

[('entrenamiento', 0.8511227369308472),
 ('imperio', 0.850968062877655),
 ('aprendiz', 0.8424110412597656),
 ('salida', 0.8375244140625),
 ('cueva', 0.8344074487686157),
 ('aprender', 0.8311670422554016),
 ('poderoso', 0.830485999584198),
 ('cosas', 0.8290493488311768),
 ('destino', 0.8284888863563538),
 ('chewbacca', 0.8283873796463013)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(negative=["fuerza"])

[('manos', 0.31187301874160767),
 ('escudos', 0.08105242252349854),
 ('torre', 0.049830615520477295),
 ('dios', -0.020483875647187233),
 ('vivo', -0.07085763663053513),
 ('sucede', -0.09378395974636078),
 ('capaz', -0.12942515313625336),
 ('quietos', -0.1389489620923996),
 ('kenobi', -0.34305277466773987),
 ('sistema', -0.345783531665802)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["velocidad"])

[('pasar', 0.9268184304237366),
 ('entiendo', 0.9103265404701233),
 ('salido', 0.9065259099006653),
 ('veo', 0.8747021555900574),
 ('listos', 0.8517149686813354),
 ('despegar', 0.8306588530540466),
 ('posicion', 0.820294976234436),
 ('plan', 0.8064688444137573),
 ('suerte', 0.7972614169120789),
 ('abran', 0.7768471837043762)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model):
    num_dimensions = 2

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

## Dataset

Se genero un dataset a partir de las de Star Wars, para el análisis se utilizaron solamente las primeras 3 para detectar relaciones entre los vínculos que se generaron en las palabras y la historia de las películas.

Se realizando la limpieza de los caracteres especiales y filtrado de las stopword en español.

## Análisis de similitudes en espacio de embeddings

Realizando varios análisis se puede observar que:
Si se busca las similitudes positivas con "skywalker" y "yoda", las primeras 5 son:
``````
['jedi', 'aprendiz', 'fuerza', 'entrenamiento' y 'sentido']
``````
Lo cual tiene mucho sentido por la historia de las películas.

Otra relación interesante que se puede buscar es entre "luke","darth" y "vader" en la cual las 2 similitudes más grandes son  'trampa' y 'padre'.

Con las similitudes negativas se puede observar que la primera para 'Luke' es 'mano', la cual pierde al final de una de las películas. Esto puede ser solo una coincidencia solamente.

## Análisis grafico

Realizando un análisis del grafico se puede observar cómo se generan nubes de puntos de palabras que se utilizan en el mismo contexto, algunos ejemplos son:

*   Velocidad, luz, halcón (nave espacial).
*   Lider, rojo, jefe. El nombre en código del **jefe** de los pilotos rebeldes es **Lider rojo**.

*   Planos, estrella y muerte.
*   Yoda, maestro, fuerza y Jedi.

Otro característica que se puede observar en la nube de palabras es la agrupación de todos los nombre de un personaje juntos, como:

*   Obi, wan, kenobi
*   Darth, Vader
*   Jabba, hutt









